# 「Ray Tuneを用いたハイパーパラメータチューニング」

【原題】Hyperparameter tuning with Ray Tune

【原著】

【元URL】https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html

【翻訳】電通国際情報サービスISID HCM事業部　櫻井 亮佑

【日付】2020年1月30日

【チュトーリアル概要】

ハイパーパラメータのチューニングを行うことで、モデルの性能を高めることが可能です。

学習率やネットワーク層のサイズを変更するだけでモデルの性能が劇的に変わることが、しばしあります。

そして幸いなことに、最善のパフォーマンスをもたらすパラメータの組み合わせを探索する際に役立つツールがあります。

[Ray Tune](https://docs.ray.io/en/latest/tune.html) は、分散型ハイパーパラメータチューニングを目的とした標準的なツールです。

Ray Tuneは最新のハイパーパラメータの探索アルゴリズムを網羅しており、TensorBoardやその他の分析ライブラリと統合されています。

また[Rayの分散型機械学習エンジン](https://ray.io/)により、分散型訓練をネイティブサポートしています。



本チュートリアルでは、PyTorchの訓練ワークフローにRay Tuneを統合する方法を解説します。

具体的には、[PyTorchのドキュメンテーションに存在するこちらのチュートリアル](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)（日本語版は[こちら](https://colab.research.google.com/github/YutaroOgawa/pytorch_tutorials_jp/blob/main/notebook/1_Learning%20PyTorch/1_4_cifar10_tutorial_jp.ipynb)）を拡張し、CIFAR10の画像分類器の訓練を行います。

以降で確認していきますが、Ray Tuneをコードに統合するには少し手を加える必要があります。

特に以下の変更を行います。

1. データ読み込みと訓練のコードを関数にラップする
2. ネットワークのパラメーターの一部を設定可能なパラメーターに変更する
3. （任意で）チェックポイントを追加する
4. モデルチューニングのための探索空間を定義する

本チュートリアルを実行するには、以下のパッケージがインストールされていることを確認してください。
-  `ray[tune]`: 分散型ハイパーパラメータチューニングのライブラリ
-  `torchvision`: データをトランスフォームするため

## セットアップ / インポート

インポートから始めましょう。

In [1]:
%matplotlib inline

In [2]:
!pip install ray
!pip install tensorboardX

     |████████████████████████████████| 47.5MB 65kB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
     |████████████████████████████████| 81kB 13.6MB/s 
     |████████████████████████████████| 1.3MB 52.8MB/s 
     |████████████████████████████████| 81kB 13.6MB/s 
     |████████████████████████████████| 204kB 58.9MB/s 
     |████████████████████████████████| 3.2MB 55.9MB/s 
     |████████████████████████████████| 133kB 52.9MB/s 
     |████████████████████████████████| 61kB 10.7MB/s 
     |████████████████████████████████| 143kB 56.7MB/s 
     |████████████████████████████████| 296kB 61.2MB/s 
     |████████████████████████████████| 102kB 14.2MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp36-none-any.whl size=12622 sha256=ee7050ceeaba250bb0bed12f19c925ddcf1918fa48aa468fc8b8459e0d6ca0b0
  Stored in directory: /root/.cache/pip/wheels/48/b4/d5/fb5b7f1d040f2ff20687e3bad6867d63155dbde5a7c10f4293
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl s

In [3]:
from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

上記のインポートのほとんどはPyTorchのモデルを構築するためのものです。
ただし、最後の3つだけはRay Tuneに必要なインポートです。

## データローダー

データローダーを関数でラップし、グローバル変数であるデータディレクトリを引数に与えます。

これにより、異なる試行間でも、データディレクトリを共有できます。

In [4]:
def load_data(data_dir="./data"):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True, download=True, transform=transform)

    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False, download=True, transform=transform)

    return trainset, testset

## 設定可能なニューラルネットワーク

設定可能なパラメーターのみをチューニングすることが可能です。

本チュートリアルでは、全結合層のサイズを設定可能なパラメータとします。

In [5]:
class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## 訓練の関数

ここから徐々に面白くなってきます。

[PyTorchのドキュメンテーション](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)（日本語版は[こちら](https://colab.research.google.com/github/YutaroOgawa/pytorch_tutorials_jp/blob/main/notebook/1_Learning%20PyTorch/1_4_cifar10_tutorial_jp.ipynb)）のサンプル例に少し手を加えます。

訓練用のスクリプトを関数`train_cifar(config, checkpoint_dir=None, data_dir=None)`でラップします。

ご想像の通り、`config`パラメータは訓練に使用したいハイパーパラメータを受け取ります。


また、`checkpoint_dir`パラメータはチェックポイントを復元するために使用されます。

そして`data_dir`にはデータを読み込み、格納するディレクトリを指定することで、複数の実行で同一のデータソースを共有できるようにします。

<code>

    net = Net(config["l1"], config["l2"])

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

</code>

オプティマイザーの学習率も設定可能なパラメータにします。

<code>
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)
</code>


また、訓練するデータを訓練用と検証用のサブセットに分割します。

具体的には、データの80%で訓練を行い、残りの20%の検証用サブセットで損失を算出します。

なお、訓練用と検証用のサブセットを反復する際に使用するバッチサイズも設定可能なパラメータにします。

## DataParallelを用いた（マルチ）GPUサポートの追加

画像分類タスクを取り組むにあたっては、GPUから多大な恩恵を受けることが可能です。
幸運なことに、Ray Tune内ではPyTorchの抽象クラスをそのまま使用できます。
したがって、モデルを`nn.DataParallel`内にラップすることで、複数のGPU上でのデータ並列訓練も実現可能です。

<code>

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

</code>

`device`変数を使用することで、GPUが利用できない環境でも訓練が行えるようにしておきます。
PyTorchでは下記のように、データを明示的にGPUのメモリ上に送る必要があります。

<code>

     for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

</code>


これでCPU上、単一のGPU上、そして複数のGPU上で訓練を行うコードができました。

また特筆すべき点として、Rayは[断片的なGPUの使用（Fractional GPUs）](https://docs.ray.io/en/master/using-ray-with-gpus.html#fractional-gpus)もサポートしており、
モデルがGPUのメモリに収まる限りにおいて、複数の試行間でGPUを共有することが可能です。
この点については、後ほど触れます。

## Ray Tuneとの連携

Ray Tuneとの連携が最も興味深い部分です。

<code>

    with tune.checkpoint_dir(epoch) as checkpoint_dir:
        path = os.path.join(checkpoint_dir, "checkpoint")
        torch.save((net.state_dict(), optimizer.state_dict()), path)

    tune.report(loss=(val_loss / val_steps), accuracy=correct / total)

</code>

ここで、始めにチェックポイントを保存し、Ray Tuneに一部の指標を報告します。

具体的には、検証用セットでの損失と正確度をRay Tuneに送ります。


Ray Tuneはこれらの指標を使って、どのハイパーパラメータの設定が最良の結果につながるか判断します。

またこれらのチェックポイントの指標は、試行を行う上でリソースの浪費を避けるため、パフォーマンスの悪い試行を早期に打ち切る際に使用されます。

なお、チェックポイントの保存は任意ですが、[Population Based Training](https://docs.ray.io/en/master/tune/tutorials/tune-advanced-tutorial.html)などの発展的なスケジューラーを使用する場合はチェックポイントの保存が必須です。

またチェックポイントを保存することで、後に訓練済みモデルを読み込み、テストセットに対して検証を行うことが可能になります。

## 訓練関数の全量

サンプルのコードの全量は以下のとおりです。

In [6]:
def train_cifar(config, checkpoint_dir=None, data_dir=None):
    net = Net(config["l1"], config["l2"])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader = torch.utils.data.DataLoader(
        train_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)

    for epoch in range(10):  # データセットに対して複数回ループします。
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # 入力の取得。データは[inputs, labels]のリスト
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # パラメーターの勾配をゼロ化
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # 統計情報の出力
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # 2000個のミニバッチ毎に出力
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0

        # 検証損失
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((net.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
    print("Finished Training")

以前からの変化が確認できるよう、コードのほとんどが紹介した元のサンプルコードから直接借用しています。

## テストセットの正確度

一般的に機械学習モデルのパフォーマンスを求めるために、モデルの訓練に使用されていない、ホールドアウトされた検証セットに対して検証を行います。

検証のコードも関数でラップします。

In [7]:
def test_accuracy(net, device="cpu"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

この関数は`device`パラメーターも引数に与えているため、GPU上でテストセットの検証を行えます。

## 探索空間の設定

最後に、Ray Tuneの探索空間を定義する必要があります、
以下のコードが一例です。

In [8]:
config = {
    "l1": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16])
}

`tune.sample_from()`関数を使用することで、ハイパーパラメータを得る独自のサンプリングメソッドを定義できるようになります。<br>
本チュートリアルでは、`l1`、`l2`パラメータが4から256の間の2のべき乗になるようにしています。

つまり、4、8、16、32、64、128、または256が設定されます。

`lr`（学習率）は、0.0001から0.1の間で一様にサンプルされます。

最後に、バッチサイズは2、4、8、そして16から選択されます。

Ray Tuneは、各試行において、上記の探索空間からランダムにパラメータの組み合わせをサンプルします。

そして、いくつかモデルを並列的に訓練し、試行した中で最もパフォーマンスに優れたものを見つけ出します。

なお、今回は悪いパフォーマンスの試行を早期に打ち切る`ASHAScheduler`も使用しています。

`train_cifar`関数を`functools.partial`でラップし、定数である`data_dir`パラメータを設定します。

また以下のように、各試行においてどのリソースが利用できるかについて、Ray Tuneに判別させることも可能です。

<code>

    gpus_per_trial = 2
    # ...
    result = tune.run(
        partial(train_cifar, data_dir=data_dir),
        resources_per_trial={"cpu": 8, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter,
        checkpoint_at_end=True)

<code>


CPUの数も指定可能であり、このパラメータは、例えばPyTorchの`DataLoader`インスタンスの`num_workers`を増やすために利用できます。

選択されたGPU数は、各試行においてPyTorchが確認できるようになります。

また、試行は当該試行に要求されていないGPUを使用することはできません。したがって、同一のリソースセットを用いた2つの試行について特に気をつける点はありません。

ここでは断片的にGPUを指定することも可能であり、`gpus_per_trial=0.5`といった指定も有効です。

これにより複数の試行は各試行同士でGPUを共有します。
なお、モデルがGPUのメモリに収まるようにする点だけは留意してください。

モデルの訓練後は、最良のパフォーマンスを行う試行を見つけ、チェックポイントのファイルから訓練されたネットワークを読み込みます。

そして、検証セットの正確度を測定し、すべてのレポートを出力します。

`main`関数は以下の通りです。

(日本語訳注：以下のmainの実行にはまずまずの時間がかかります。30分程度)


In [9]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    data_dir = os.path.abspath("./data")
    load_data(data_dir)
    config = {
        "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16])
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        partial(train_cifar, data_dir=data_dir),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    best_trained_model = Net(best_trial.config["l1"], best_trial.config["l2"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_acc = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {}".format(test_acc))


if __name__ == "__main__":
    # ここで試行毎に使用するGPU数を変更できます
    #main(num_samples=10, max_num_epochs=10, gpus_per_trial=0)
    main(num_samples=10, max_num_epochs=10, gpus_per_trial=1)


Extracting /content/data/cifar-10-python.tar.gz to /content/data
Files already downloaded and verified


2021-02-15 23:46:26,218	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265


2021-02-15 23:46:29,216	WARNING experiment.py:292 -- No name detected on trainable. Using DEFAULT.
2021-02-15 23:46:29,217	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.


== Status ==
Memory usage on this node: 1.3/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2/2 CPUs, 1/1 GPUs, 0.0/7.32 GiB heap, 0.0/2.49 GiB objects (0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/DEFAULT_2021-02-15_23-46-29
Number of trials: 1/10 (1 RUNNING)
+---------------------+----------+-------+--------------+------+------+-----------+
| Trial name          | status   | loc   |   batch_size |   l1 |   l2 |        lr |
|---------------------+----------+-------+--------------+------+------+-----------|
| DEFAULT_06375_00000 | RUNNING  |       |            8 |  256 |   16 | 0.0148522 |
+---------------------+----------+-------+--------------+------+------+-----------+


(pid=227) Files already downloaded and verified
(pid=227) Files already downloaded and verified
(pid=227) [1,  2000] loss: 2.040
(pid=227) [1,  4000] loss: 0.932
Result for DEFAULT_06375_00000:
  accur

2021-02-16 00:12:53,171	INFO tune.py:450 -- Total run time: 1588.07 seconds (1583.79 seconds for the tuning loop).


Result for DEFAULT_06375_00009:
  accuracy: 0.1209
  date: 2021-02-16_00-12-53
  done: true
  experiment_id: 6198037d504a495c81dc8518caa8d437
  hostname: d1d37fbe0b7b
  iterations_since_restore: 1
  loss: 2.2997427158355714
  node_ip: 172.28.0.2
  pid: 3621
  should_checkpoint: true
  time_since_restore: 24.84288239479065
  time_this_iter_s: 24.84288239479065
  time_total_s: 24.84288239479065
  timestamp: 1613434373
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 06375_00009
  
== Status ==
Memory usage on this node: 2.8/12.7 GiB
Using AsyncHyperBand: num_stopped=10
Bracket: Iter 8.000: -1.4005895887613296 | Iter 4.000: -1.4330047328472137 | Iter 2.000: -1.50707006316185 | Iter 1.000: -1.942865083360672
Resources requested: 2/2 CPUs, 1/1 GPUs, 0.0/7.32 GiB heap, 0.0/2.49 GiB objects (0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/DEFAULT_2021-02-15_23-46-29
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+---------------------+------------+--------------

コードを実行すると、以下のような出力が得られます。

（日本語訳注：細かな値などは実行ごとに変わる可能性があります）

<code>

    Number of trials: 10 (10 TERMINATED)
    +-----+------+------+-------------+--------------+---------+------------+--------------------+
    | ... |   l1 |   l2 |          lr |   batch_size |    loss |   accuracy | training_iteration |
    |-----+------+------+-------------+--------------+---------+------------+--------------------|
    | ... |   64 |    4 | 0.00011629  |            2 | 1.87273 |     0.244  |                  2 |
    | ... |   32 |   64 | 0.000339763 |            8 | 1.23603 |     0.567  |                  8 |
    | ... |    8 |   16 | 0.00276249  |           16 | 1.1815  |     0.5836 |                 10 |
    | ... |    4 |   64 | 0.000648721 |            4 | 1.31131 |     0.5224 |                  8 |
    | ... |   32 |   16 | 0.000340753 |            8 | 1.26454 |     0.5444 |                  8 |
    | ... |    8 |    4 | 0.000699775 |            8 | 1.99594 |     0.1983 |                  2 |
    | ... |  256 |    8 | 0.0839654   |           16 | 2.3119  |     0.0993 |                  1 |
    | ... |   16 |  128 | 0.0758154   |           16 | 2.33575 |     0.1327 |                  1 |
    | ... |   16 |    8 | 0.0763312   |           16 | 2.31129 |     0.1042 |                  4 |
    | ... |  128 |   16 | 0.000124903 |            4 | 2.26917 |     0.1945 |                  1 |
    +-----+------+------+-------------+--------------+---------+------------+--------------------+


    Best trial config: {'l1': 8, 'l2': 16, 'lr': 0.00276249, 'batch_size': 16, 'data_dir': '...'}
    Best trial final validation loss: 1.181501
    Best trial final validation accuracy: 0.5836
    Best trial test set accuracy: 0.5806
</code>

ほとんどの試行は、リソースの浪費を避けるために早々に打ち切られています。

最良のパフォーマンスを発揮した試行は、検証セットを使用して約58%の正確度を達成しています。

以上、Ray Tuneを用いたハイパーパラメータチューニングを解説しました。

本手法により、自身で構築したPyTorchのモデルのハイパーパラメータをチューニングすることができます。